In [1]:
import mxnet as mx

def get_iterators(batch_size, data_shape=(3, 224, 224)):
    train = mx.io.ImageRecordIter(
#         path_imgrec         = '/data4/srip_face/img/jump_detector/old/jump_train_old.rec', 
        path_imgrec         = '/data4/srip_face/img/jump_detector/jump_train.rec', 
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        shuffle             = True,
        rand_crop           = True,
        rand_mirror         = True)
    val = mx.io.ImageRecordIter(
#         path_imgrec         = '/data4/srip_face/img/jump_detector/old/jump_valid_old.rec',
        path_imgrec         = '/data4/srip_face/img/jump_detector/jump_valid.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        rand_crop           = False,
        rand_mirror         = False)
    return (train, val)

In [2]:
import os, urllib
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.urlretrieve(url, filename)
        
def get_model(prefix, epoch):
    download(prefix+'-symbol.json')
    download(prefix+'-%04d.params' % (epoch,))

get_model('http://data.mxnet.io/models/imagenet/caffenet/caffenet', 0)
sym, arg_params, aux_params = mx.model.load_checkpoint('caffenet', 0)

In [3]:
def get_fine_tune_model(symbol, arg_params, num_classes, layer_name='drop7'):
    """
    symbol: the pre-trained network symbol
    arg_params: the argument parameters of the pre-trained model
    num_classes: the number of classes for the fine-tune datasets
    layer_name: the layer name before the last fully-connected layer
    """
    all_layers = symbol.get_internals()
    net = all_layers[layer_name+'_output']
    net = mx.symbol.FullyConnected(data=net, num_hidden=num_classes, name='fc8')
    net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
    new_args = dict({k:arg_params[k] for k in arg_params if 'fc8' not in k})
    return (net, new_args)

In [4]:
import logging
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

def fit(symbol, arg_params, aux_params, train, val, batch_size, num_gpus=1, num_epoch=1):
    devs = [mx.gpu(i) for i in range(num_gpus)] # replace mx.gpu by mx.cpu for CPU training
    mod = mx.mod.Module(symbol=symbol, context=devs)
    mod.bind(data_shapes=train.provide_data, label_shapes=train.provide_label)
    mod.init_params(initializer=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=1))
    mod.set_params(arg_params, aux_params, allow_missing=True)
    
    mod.fit(train, val, 
        num_epoch=num_epoch,
        batch_end_callback = mx.callback.log_train_metric(100,True),
        epoch_end_callback = mx.callback.do_checkpoint("caffenet-mxnet-jump-start-bootstrap", 1),
        kvstore='device',
        optimizer='sgd',
        optimizer_params={'learning_rate':0.0009},
        eval_metric='acc')
#        eval_metric=mx.metric.CrossEntropy())
    
    return mod

In [5]:
num_classes = 2 # This is binary classification
batch_per_gpu = 128
num_gpus = 1
epoch = 10
(new_sym, new_args) = get_fine_tune_model(sym, arg_params, num_classes)

batch_size = batch_per_gpu * num_gpus
(train, val) = get_iterators(batch_size)
mod = fit(new_sym, new_args, aux_params, train, val, batch_size, num_gpus, epoch)
metric = mx.metric.Accuracy()
mod_score = mod.score(val, metric)
print mod_score

2017-09-14 10:58:42,616 Already bound, ignoring bind()
/home/srip_face/.local/lib/python2.7/site-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
2017-09-14 10:58:47,265 Iter[0] Batch[0] Train-accuracy=0.273438
2017-09-14 10:59:09,130 Iter[0] Batch[100] Train-accuracy=0.799687
2017-09-14 10:59:30,972 Iter[0] Batch[200] Train-accuracy=0.827031
2017-09-14 10:59:36,644 Epoch[0] Train-accuracy=0.823618
2017-09-14 10:59:36,646 Epoch[0] Time cost=54.001
2017-09-14 10:59:40,686 Saved checkpoint to "caffenet-mxnet-jump-start-bootstrap-0001.params"
2017-09-14 10:59:44,244 Epoch[0] Validation-accuracy=0.788352
2017-09-14 10:59:44,354 Iter[1] Batch[0] Train-accuracy=0.804688
2017-09-14 11:00:06,213 Iter[1] Batch[100] Train-accuracy=0.837344
2017-09-14 11:00:28,067 Iter[1] Batch[200] Train-accuracy=0.843047
2017-09-14 11:00:33,740 Epoch[1] Train-accuracy=0.8449

[('accuracy', 0.845407196969697)]


In [ ]:
prefix = 'caffenet-mxnet-jump-start-128-bootstrap'
mc = mod.save_checkpoint(prefix, epoch)